In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install seaborn

Importaciones

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Carga de los datos y filtrado

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Apartado 1
En el primer apartado, comenzamos con un EDA o Análisis Exploratorio de los Datos que nos servirá para comprender mejor el problema al que nos enfrentamos

In [ ]:
# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Número de instancias y características
print(f'Número de instancias: {wind_ava.shape[0]}')
print(f'Número de características: {wind_ava.shape[1]}')

# Variables categóricas y numéricas
categorical_vars = wind_ava.select_dtypes(include=['object']).columns
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns
print(f'Variables categóricas: {categorical_vars}')
print(f'Variables numéricas: {numerical_vars}')

# Comprueba si hay valores faltantes y qué variables los tienen
missing_values = wind_ava.isnull().sum()
print(f'Valores faltantes por variable:\n{missing_values[missing_values > 0]}')

# Columnas constantes
constant_columns = [col for col in wind_ava.columns if wind_ava[col].nunique() <= 1]
print(f'Columnas constantes: {constant_columns}')

# Elimina las columnas constantes
wind_ava = wind_ava.loc[:, wind_ava.apply(pd.Series.nunique) != 1]


Por lo tanto, estamos ante un problema de ``regresión`` (al ser valores continuos) y en el cual tenemos ``4748 instancias``. Cada instancia a su vez presenta 22 variables númericas, 1 variable 'datetime' y una variable objetivo 'energy'. Cabe destacar además que el conjunto de datos no presenta valores faltantes ni columnas que sean constantes, por lo que no hay que hacer ningún tipo de preproceso relativo a esos problemas.

### Matriz de correlación
Podemos realizar una matriz de correlación para comprobar qué variables están fuertemente relacionadas:

In [ ]:
# Realizamos un analisis de la correlación de las variables númericas
correlation = wind_ava[numerical_vars].corr()
plt.figure(figsize=(20, 8))
sns.heatmap(correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

Con estos resultados podemos ver que tanto las variables `` lai_lv.13`` y `` lai_hv.13 ``, como las variables `` v10.13 `` y `` v10n.13``, tienen una correlacion de aproximadamente 1. Por lo tanto, podemos quitar una de cada pareja, ya que dan información redundante.

In [ ]:
# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Realizamos un analisis de la correlación de las variables númericas
correlation = wind_ava[numerical_vars].corr()
plt.figure(figsize=(20, 8))
sns.heatmap(correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

Podemos observar que sigue habiendo variables muy correlacionadas entre ellas, llegando a haber resultados de 0.99, pero en ningún caso aparece un 1 fuera de la diagonal. Además, con esta matriz podemos ver que la variable que más correlación tiene con la energía producida es `` p59.162 `` (Vertical integral of divergence of kinetic energy) con un grado casi del 50%.


### Distribución valores variables

Con el próposito de ver como se distribuyen los datos en nuestras variables hemos creado los siguientes histogramas:

In [ ]:
import numpy as np

# Número de variables numéricas
num_vars = len(numerical_vars)

# Calcula el número de filas y columnas para los subplots
num_cols = 4  # número máximo de columnas
num_rows = np.ceil(num_vars / num_cols).astype(int)

# Crea una figura y un conjunto de subtramas
fig, axs = plt.subplots(num_rows, num_cols, figsize=(6*num_cols, 4*num_rows))

# Aplana el array de subplots para facilitar su recorrido
axs = axs.flatten()

# Para cada variable numérica, crea un histograma en su respectivo subplot
for i, col in enumerate(numerical_vars):
    axs[i].hist(wind_ava[col], bins=30, alpha=0.5, label=col)
    axs[i].set_title(f'Distribución de {col}')
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Frecuencia')
    axs[i].legend(loc='upper right')

# Elimina los subplots vacíos si el número de variables no es múltiplo de 4
if num_vars % num_cols != 0:
    for j in range(i+1, num_rows*num_cols):
        fig.delaxes(axs[j])

# Ajusta el espacio entre los subplots
plt.tight_layout()

# Muestra la figura con todos los histogramas
plt.show()

Estos histogramas nos ayudan a conocer mejor como se distribuyen los datos  y qué valores mínimos y máximos toman nuestras variables.

### Energía producida por meses

Si realizamos la media por meses de la energía producida y mostramos un gráfico de diferentes años podemos hacernos una idea de en qué meses se produce más energía:

In [ ]:
# Se asegura de que 'datetime' es de tipo datetime
wind_ava['datetime'] = pd.to_datetime(wind_ava['datetime'])

# Lista de colores para cada año
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Iterar sobre cada año único en los datos
for year, color in zip(wind_ava['datetime'].dt.year.unique(), colors):
    # Filtra los datos para el año actual
    filtered_data = wind_ava[wind_ava['datetime'].dt.year == year]
    
    # Agrupa por mes y calcula la media de 'energy' para el año actual
    monthly_energy = filtered_data.groupby(filtered_data['datetime'].dt.to_period('M')).mean()
    
    # Crea un gráfico para el año actual
    plt.figure(figsize=(12, 6))
    
    # Traza la media mensual de 'energy' para el año actual con el color correspondiente
    plt.plot(monthly_energy.index.to_timestamp(), monthly_energy['energy'], color=color)
    
    # Configura etiquetas y título
    plt.xlabel('Fecha')
    plt.ylabel('Energía Media')
    plt.title(f'Media Mensual de Energía para el Año {year}')
    
    # Muestra el gráfico
    plt.show()

Como vemos, los meses del año en los que menos energía se suele producir son julio y agosto, mientras que generalmente se produce más energía en noviembre o marzo.

### Outliers

Los outliers identifican el número de datos que están fuera de la distribución. Nos ayudan a identificar si se han producido errores de medición, anomalías o cambios en la distribución de los datos.

In [ ]:
# Para cada variable numérica, visualizar los outliers
for col in numerical_vars:
    Q1 = wind_ava[col].quantile(0.25)
    Q3 = wind_ava[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = wind_ava[(wind_ava[col] < (Q1 - 1.5 * IQR)) | (wind_ava[col] > (Q3 + 1.5 * IQR))]
    print(f"Outliers en {col}: {outliers[col].count()}")

# Lista de variables para las que visualizar outliers
variables = ['energy', 'p55.162.13', 'cape.13', 'p59.162.13', 'sp.13', 'iews.13', 'inss.13', 'fsr.13']

# Para cada variable, creamos un gráfico de caja
for var in variables:
    plt.figure(figsize=(10, 4))
    boxplot = plt.boxplot(wind_ava[var], vert=False)
    plt.setp(boxplot['boxes'], color='blue')
    plt.title(f'Boxplot de {var}')
    plt.show()

Con estos resultados podemos ver que las variables con el mayor número de outliers son ``cape.13`` e ``inss.13``, con 697 y 431 outliers respectivamente.

# Apartado 2
La evaluación ``outer`` del modelo se llevará a cabo por medio de la evualuación train/test, ya que disponemos de un gran número de datos. Por otro lado, en la evaluavión ``inner`` se utilizará más de un método de evaluación con el fin de probarlos y comparar sus resultados. Esto incluye evaluación cruzada y el uso de Grid-search (para ajustar múltiples hiperparámetros).

En cuanto a la métrica, en general hemos utilizado ``MSE`` y ``RSME``. Hemos decidido hacerlo con estas medidas ya que están en las mismas unidades que la variable objetivo, lo que facilita su interpretación. Además, tanto el MSE como el RMSE penalizan más los errores grandes que los pequeños, debido a que cada error se eleva al cuadrado antes de hacer la media.

# Apartado 3
En primer lugar, y usando el método KNN, vamos a comprobar cuál es el mejor método de escalado para nuestros datos. Para ello utilizaremos validación cruzada y nos basaremos en el error cuadrático medio negativo (o RMSE), por lo que el modelo con el resultado más pequeño será el modelo más preciso

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Quitamos "energy" de las variables numéricas, ya que querremos usarla como variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Dividir los datos en conjuntos de entrenamiento y prueba (evaluación externa)
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Definir los métodos de escalado para evaluar
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# Evauluación interna con 3-fold time series cross-validation
inner = TimeSeriesSplit(n_splits=3)

# Diccionario para almacenar los resultados de la evaluación interna
inner_scores = {}

# Pipeline para StandardScaler
knn_standard = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

scores_std = cross_val_score(knn_standard, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['StandardScaler'] = -scores_std.mean()

# Pipeline para MinMaxScaler
knn_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsRegressor())
])
scores_minmax = cross_val_score(knn_minmax, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['MinMaxScaler'] = -scores_minmax.mean()

# Pipeline para RobustScaler
knn_robust = Pipeline([
    ('scaler', RobustScaler()),
    ('knn', KNeighborsRegressor())
])
scores_robust = cross_val_score(knn_robust, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['RobustScaker'] = -scores_robust.mean()

# Imprimimos los resultados
for scaler_name, score in inner_scores.items():
    print(f"Variable Name: {scaler_name}")
    print(f"Variable Score: {score}")
    print()

# Elegimos el mejor método de escalado según el RMSE
mejor_scaler = min(inner_scores, key=inner_scores.get)
print(f"El mejor método de escalado es {mejor_scaler} con un RMSE de {inner_scores[min(inner_scores, key=inner_scores.get)]}")

# Apartado 4
De manera similar, ahora evaluaremos y mediremos los tiempos de distintos métodos. Primero lo haremos con sus hiperparámetros por defecto, como se muestra a continuación

In [ ]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor

# Crear instancias de los modelos
knn = KNeighborsRegressor()
tree = DecisionTreeRegressor()
linear = LinearRegression()
lasso = Lasso()
svm = SVR()
dummy = DummyRegressor(strategy='mean')

# Crear listas para almacenar los tiempos de entrenamiento y los errores
times = {name: [] for name in scalers}
errors = {name: [] for name in scalers}

models = [knn, tree, linear, lasso, svm, dummy]
model_names = ['KNN', 'Árboles de Regresión', 'Regresión Lineal', 'Regresión Lasso', 'SVM', 'Dummy']

# Entrenar y evaluar los modelos con diferentes métodos de escalado
for scaler_name, scaler in scalers.items():
    if scaler is not None:
        # Escala los datos de entrenamiento
        X_train_scaled = scaler.fit_transform(X_train)
        # Escala los datos de prueba
        X_test_scaled = scaler.transform(X_test)
    else:
        # Si no se utiliza escalado, usa los datos originales
        X_train_scaled = X_train
        X_test_scaled = X_test

    for model, name in zip(models, model_names):
        start_time = time.time()
        model.fit(X_train_scaled, y_train)
        end_time = time.time()
        train_time = end_time - start_time
        times[scaler_name].append(train_time)

        y_pred = model.predict(X_test_scaled)
        error = mean_squared_error(y_test, y_pred)
        errors[scaler_name].append(error)

        print(f"Modelo: {name}, Escalado: {scaler_name}")
        print(f"Tiempo de entrenamiento: {train_time} segundos")
        print(f"Error cuadrático medio: {error}")
        print("------------------------")


# Crea una figura y un conjunto de subtramas
fig, axs = plt.subplots(2, figsize=(12, 6))

# Crea una lista con los nombres de los escaladores
scaler_names = list(scalers.keys())

# Crea diagramas de barras para los tiempos de entrenamiento
for i, scaler_name in enumerate(scaler_names):
    axs[0].bar(np.arange(len(model_names)) + i*0.1, times[scaler_name], width=0.1, label=scaler_name)

axs[0].set_xticks(np.arange(len(model_names)) + len(scalers)*0.1/2)
axs[0].set_xticklabels(model_names)
axs[0].legend()
axs[0].set_title('Tiempo de entrenamiento')
axs[0].set_ylabel('Segundos')

# Crea diagramas de barras para los errores
for i, scaler_name in enumerate(scaler_names):
    axs[1].bar(np.arange(len(model_names)) + i*0.1, errors[scaler_name], width=0.1, label=scaler_name)

axs[1].set_xticks(np.arange(len(model_names)) + len(scalers)*0.1/2)
axs[1].set_xticklabels(model_names)
axs[1].legend()
axs[1].set_title('Error cuadrático medio')
axs[1].set_ylabel('Error')

# Muestra la figura
plt.tight_layout()
plt.show()


Estos gráficos nos ayudan a ver que, con los hiperparámetros por defecto, el modelo con el menor error es ``KNN``. Este modelo es también el más beneficioso en cuanto al tiempo, llegando a ser casi tan instantáneo como el "Dummy". El resto de errores, como cabe esperar, son prácticamente todos menores que el conseguido simplemente haciendo la media (en el "Dummy"). Todos excepto los obtenidos con SVM y los escaladores MinMax y Robust. Esto significa que sería preferible devolver la media antes que usar los valores devueltos por el modelo SVM con sus parámetros por omisión, sobre todo teniendo en cuenta que también es el modelo que más tarda en ejecutar.

En cuanto a los escaladores, las diferencias entre ellos son bastante pequeñas, por lo que a partir de ahora utilizaremos el ``StandardScaler``. Consideramos que es lo mejor, ya que el enunciado nos sugería que utilizaramos el mejor escalador para KNN y porque nos será beneficioso más adelante, al ser también el escalador que mejor funciona en SVM.

### GridSearch
Ahora volveremos a evaluar los modelos, pero esta vez probando distintos valores para los hiperparámetros y mediante el uso de un procedimiento sistemático como ``GridSearchCV``

In [ ]:
from sklearn.model_selection import GridSearchCV

mejor_scaler = StandardScaler()
X_train_scaled = mejor_scaler.fit_transform(X_train)
X_test_scaled = mejor_scaler.transform(X_test)

# Hiperparámetros
knn_params = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
tree_params = {
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2],
    'max_leaf_nodes': [10, 20, 30],
    'min_impurity_decrease': [0.0, 0.1],
    'criterion': ['squared_error'],
    'splitter': ['best', 'random'],
    'min_weight_fraction_leaf': [0.0, 0.5]
}
linear_params = {
    'fit_intercept': [True, False]
}
lasso_params = {
    'alpha': [0.1, 0.5, 1, 5, 10],
    'fit_intercept': [True, False],
    'max_iter': [5000, 10000],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1],
    'positive': [True, False],
    'selection': ['cyclic', 'random'],
    'random_state': [None, 100472166]
}
svm_params = {
    'C': [0.1, 1, 10, 100, 1000, 2500],
    'shrinking': [True, False],
    'kernel': ['rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'coef0': [0, 0.5, 1]
}

# Realizamos GridSearchCV para cada modelo
knn_grid = GridSearchCV(KNeighborsRegressor(), knn_params, cv=inner)
tree_grid = GridSearchCV(DecisionTreeRegressor(), tree_params, cv=inner)
linear_grid = GridSearchCV(LinearRegression(), linear_params, cv=inner)
lasso_grid = GridSearchCV(Lasso(), lasso_params, cv=inner)
svm_grid = GridSearchCV(SVR(), svm_params, cv=inner)

# Ajusta KNN con los mejores hiperparámetros
start_time = time.time()
knn_grid.fit(X_train_scaled, y_train)
end_time = time.time()
train_time = end_time - start_time

# Obtenemos los mejores hiperparámetros y puntuaciones para KNN
best_knn_params = knn_grid.best_params_
best_knn_score = knn_grid.best_score_
y_pred = knn_grid.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("KNN:")
print("Mejores Hiperparámetros:", best_knn_params)
print("Mejor Puntuación:", best_knn_score)
print("Error cuadrático medio:", mse)
print("Tiempo de entrenamiento:", train_time)
print("------------------------")

# Ajusta árboles de decisión con los mejores hiperparámetros
start_time = time.time()
tree_grid.fit(X_train_scaled, y_train)
end_time = time.time()
train_time = end_time - start_time

# Obtenemos los mejores hiperparámetros y puntuaciones para árboles de decisión
best_tree_params = tree_grid.best_params_
best_tree_score = tree_grid.best_score_
y_pred = tree_grid.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Árbol de Decisión:")
print("Mejores Hiperparámetros:", best_tree_params)
print("Mejor Puntuación:", best_tree_score)
print("Error cuadrático medio:", mse)
print("Tiempo de entrenamiento:", train_time)
print("------------------------")

# Ajusta regresión lineal con los mejores hiperparámetros
start_time = time.time()
linear_grid.fit(X_train_scaled, y_train)
end_time = time.time()
train_time = end_time - start_time

# Obtenemos los mejores hiperparámetros y puntuaciones para regresión lineal
best_linear_params = linear_grid.best_params_
best_linear_score = linear_grid.best_score_
y_pred = linear_grid.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Regresión Lineal:")
print("Mejores Hiperparámetros:", best_linear_params)
print("Mejor Puntuación:", best_linear_score)
print("Error cuadrático medio:", mse)
print("Tiempo de entrenamiento:", train_time)
print("------------------------")

# Ajusta regresuón Lasso con los mejores hiperparámetros
start_time = time.time()
lasso_grid.fit(X_train_scaled, y_train)
end_time = time.time()
train_time = end_time - start_time

# Obtenemos los mejores hiperparámetros y puntuaciones para regresión Lasso
best_lasso_params = lasso_grid.best_params_
best_lasso_score = lasso_grid.best_score_
y_pred = lasso_grid.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Lasso:")
print("Mejores Hiperparámetros:", best_lasso_params)
print("Mejor Puntuación:", best_lasso_score)
print("Error cuadrático medio:", mse)
print("Tiempo de entrenamiento:", train_time)
print("------------------------")

# Ajusta SVM con los mejores hiperparámetros
start_time = time.time()
svm_grid.fit(X_train_scaled, y_train)
end_time = time.time()
train_time = end_time - start_time

# Obtenemos los mejores hiperparámetros y puntuaciones para SVM
best_svm_params = svm_grid.best_params_
best_svm_score = svm_grid.best_score_
y_pred = svm_grid.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("SVM:") 
print("Mejores Hiperparámetros:", best_svm_params)
print("Mejor Puntuación:", best_svm_score)
print("Error cuadrático medio:", mse)
print("Tiempo de entrenamiento:", train_time)
print("------------------------")

Podemos comparar los valores que acabamos de obtener y los valores de los modelos por omisión mediante el siguiente gráfico:

In [ ]:
# Crea una figura con un tamaño específico (ancho, alto)
fig, ax = plt.subplots(figsize=(15, 7))

# Crea una lista con los nombres de los modelos
model_names = ['KNN', 'Árbol de Decisión', 'Regresión Lineal', 'Lasso', 'SVM', 'Dummy']

# Cre listas para almacenar las puntuaciones y los errores
scores = [best_knn_score, best_tree_score, best_linear_score, best_lasso_score, best_svm_score]
errors_now = [mean_squared_error(y_test, knn_grid.predict(X_test_scaled)),
          mean_squared_error(y_test, tree_grid.predict(X_test_scaled)),
          mean_squared_error(y_test, linear_grid.predict(X_test_scaled)),
          mean_squared_error(y_test, lasso_grid.predict(X_test_scaled)),
          mean_squared_error(y_test, svm_grid.predict(X_test_scaled)),
          errors['StandardScaler'][5]]

# Crea diagramas de barras para los errores
bar_width = 0.35
index = np.arange(len(model_names))

bar1 = ax.bar(index, errors_now, bar_width, color='blue', label='MSE con ajuste')
bar2 = ax.bar(index + bar_width, errors['StandardScaler'], bar_width, color='orange', label='MSE por defecto')

ax.set_xlabel('Modelos')
ax.set_ylabel('Error')
ax.set_title('Comparación de errores')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(model_names)
ax.legend()

# Ajusta el diseño y muestra la figura
plt.tight_layout()
plt.show()

Por lo tanto, y basándonos en los resultados obtenidos hasta el momento, llegamos a las siguientes conclusiones:

Una vez hecho el ajuste de hiperparámetros ``SVM`` obtuvo la puntuación más alta, con un 0.65, lo que indica que ahora es el modelo que mejor se ajusta a nuestros datos. Y es que el error cuadrático medio es mucho menor que cuando se usaban los parámetros por defecto, llegando a bajar de 427590 a 159220. Por todas estas razones consideramos que SVM es el mejor modelo para usar con los datos proporcionados, incluso habiendo sido el peor en las evaluaciones anteriores.

El ajuste de hiperparámetros ha sido por lo tanto una gran ayuda a la hora de obtener valores más precisos, no sólo para SVM, sino también para el resto de modelos. Y es que hay modelos que han reducido poco su error (como KNN), otros que lo han reducido significativamente (como árboles de decisión y SVM) y el resto se han mantenido igual (regresión lineal y regresión Lasso).

En cuanto a velocidad, el método más rápido obviamente es el "Dummy" con 0.0 segundos (es decir, es un método prácticamente instantáneo para este conjunto de datos). Sin embargo, los resultados del resto de métodos son obviamente mejores que los obtenidos con un "DummyRegressor", lo que indica que cualquier modelo de los propuestos es más útil que simplemente asumir el valor medio.

En general, existe un equilibrio entre el tiempo de ejecución y la mejora de resultados. Por ejemplo, el modelo SVM es el que más tarda en entrenar, pero al mismo tiempo es el que mejor resultado presenta. En cambio, la regresión lineal tarda menos de 1 segundo en ajustarse pero eso provoca que tenga un MSE bastante elevado, además de ser idéntico al que obtuvo con sus valores por omisión.

Por último, nos parece importante destacar que hemos llegado a este resultado en parte gracias al preproceso y al análisis de las variables que realizamos. Gracias a él por ejemplo pudimos eliminar dos variables con una alta correlacción, y si hubiera habido valores nulos o columnas de muy baja correlacción con la variable objetivo también nos habría ayudado. Algunos modelos (como los árboles de decisión) también proporcionan información sobre la importancia de los atributos. Por ejemplo, se puede obtener la importancia de los atributos en uno de estos árboles mediante el atributo feature_importances_.